In [1]:
import numpy as np
import locale
#set locale (for currency formatting)
locale.setlocale(locale.LC_ALL, 'en_US.utf8')

'en_US.utf8'

In [2]:
#define a seed for the random number generator (to ensure that results are reproducible)
np.random.seed(367)
# now what is the role of this statement........
# now in the code below we are using pseudo-random generator...so there is a possibility that
# next time u run this whole notebook u may get different ans...so to prevent this from happening we use seed


In [3]:
class Advertising_Campaign():
    def __init__(self,campaign_id):
        self.id = campaign_id
        #assign a random conversion rate between 1% and 20% for this campaign
        self.conversion_rate = np.random.uniform(0.01, 0.2)
        #assign a random amount of profit per successful sale for this campaign
        self.average_profit_per_sale = np.random.uniform(100., 200.)
        #track the number of successes and failures
        self.sales = 0
        self.no_sales = 0 # this is no sales...not number of sales...
        # so now... i will be declaring some convenient functions
        #a function that returns the total number of times this campaign has been tried
    def total_trials(self):
        return self.sales + self.no_sales
  
      #a function that returns the total profit thus far generated by this campaign
    def total_profit(self):
        return self.sales * self.average_profit_per_sale

      #a function that returns the actual amount of profit per trial
    def actual_profit_per_trial(self):
        if self.total_trials() > 0:
            return self.total_profit() / self.total_trials()
        else:
            return 0.

      # a function that returns the expected (theoretical) amount of profit per trial
      #  we will compare our output...with this theoretical value      
    def expected_profit_per_trial(self):
        return self.conversion_rate * self.average_profit_per_sale
    
def try_campaign(campaign):
    if np.random.random() <= campaign.conversion_rate:
        campaign.sales += 1
    else:
        campaign.no_sales += 1   

In [4]:
##### INITIALIZE THE ENVIRONMENT #####
#define number of advertising campaigns
number_of_advertising_campaigns = 8

#define number of customers
number_of_customers = 1000000

#define a list to hold the advertising campaign objects
campaigns = []

#create all of the advertising campaigns
for c in range(number_of_advertising_campaigns):
    campaigns.append(Advertising_Campaign(c))
    #print the expected profit per trial for the newly created campaign
    print('Campaign {0}: Expected profit per trial = {1}'.format(campaigns[c].id, 
                                                               locale.currency(campaigns[c].expected_profit_per_trial(), grouping=True)))

Campaign 0: Expected profit per trial = $3.07
Campaign 1: Expected profit per trial = $3.10
Campaign 2: Expected profit per trial = $2.67
Campaign 3: Expected profit per trial = $35.89
Campaign 4: Expected profit per trial = $6.30
Campaign 5: Expected profit per trial = $11.01
Campaign 6: Expected profit per trial = $14.87
Campaign 7: Expected profit per trial = $3.91


# THOMAS SAMPLING SIMULATION

In [5]:
for customer in range(number_of_customers):
    #define variables to keep track of the best beta value and the campaign to try for this customer
    index_of_campaign_to_try = -1
    best_beta_value = -1
    for campaign in campaigns:
        #get the current beta value for this campaign (since we're using Thompson Sampling)
        # now why did we divide it by 2.....this comes with experience
        current_campaign_beta_value = np.random.beta(campaign.actual_profit_per_trial() + 1., number_of_advertising_campaigns / 2.)
        if current_campaign_beta_value>best_beta_value:
            #update the best beta value
            best_beta_value = current_campaign_beta_value
            #update the index of the campaign to try for this customer
            index_of_campaign_to_try = campaign.id
            
    #try the chosen advertising campaign on this customer
    try_campaign(campaigns[index_of_campaign_to_try])
        
#define variables to hold total profits for the Thompson Sampling and Uniform Sampling methods
total_profit_thompson_sampling = 0
total_profit_uniform_sampling = 0

#compute the number of customers that would see each advertising campaign if we had used a uniform sampling approach
uniform_customers_per_campaign = number_of_customers / number_of_advertising_campaigns
#display profits and number of trials for each campaign, and compute total profits for both thompson and uniform sampling
for campaign in campaigns:
    print('Campaign {0}: Actual profit per trial = {1}, Total trials = {2}'.format(campaign.id, 
                                                                                 locale.currency(campaign.actual_profit_per_trial(), grouping=True), 
                                                                                 campaign.total_trials()))
    
    # now in our 8 campaign object...profits are already measured....so here were just iterating to get the final result.    
    total_profit_thompson_sampling += campaign.total_profit()
    #   here we are using theoretical value to determine this......  
    total_profit_uniform_sampling += (uniform_customers_per_campaign * campaign.conversion_rate * campaign.average_profit_per_sale)

    
#print overall results
print('\nThompson Sampling total profit: {0}'.format(locale.currency(total_profit_thompson_sampling, grouping=True)))
print('Uniform Sampling total profit: {0}'.format(locale.currency(total_profit_uniform_sampling, grouping=True)))
print('Thompson Sampling absolute improvement: {0}'.format(locale.currency(total_profit_thompson_sampling - total_profit_uniform_sampling, grouping=True)))
print('Thompson Sampling relative improvement: {0:.2%}'.format((total_profit_thompson_sampling / total_profit_uniform_sampling) - 1.))


Campaign 0: Actual profit per trial = $2.18, Total trials = 830
Campaign 1: Actual profit per trial = $1.87, Total trials = 1371
Campaign 2: Actual profit per trial = $3.50, Total trials = 2404
Campaign 3: Actual profit per trial = $36.06, Total trials = 801568
Campaign 4: Actual profit per trial = $5.90, Total trials = 7773
Campaign 5: Actual profit per trial = $10.95, Total trials = 61989
Campaign 6: Actual profit per trial = $14.73, Total trials = 117101
Campaign 7: Actual profit per trial = $4.00, Total trials = 6964

Thompson Sampling total profit: $31,397,589.59
Uniform Sampling total profit: $10,102,387.69
Thompson Sampling absolute improvement: $21,295,201.90
Thompson Sampling relative improvement: 210.79%
